In [1]:
import pandas as pd
import numpy as np
import os
import pickle

# Define lists of positions

## Define DR positions

In [12]:
h37rv_summary_df = pd.read_csv("/n/data1/hms/dbmi/farhat/mtb_data/mycobrowser/h37rv_mycobrowser_genome_summary.csv")
h37rv_summary_df.head()

,gene_id,gene_name,chrom_start,chrom_stop,strand,gene_category_vargas2021,feature,function,product,functional_category,PFAM,gene_ontology,dr_mutations
0,Rv0001,dnaA,1,1524,+,Essential,CDS,Plays an important role in the initiation and ...,Chromosomal replication initiator protein DnaA,information pathways,P49993,"GO:0003688,GO:0006270,GO:0005737,GO:0017111,GO...",NaN
1,Rv0002,dnaN,2052,3260,+,Non-Essential,CDS,"DNA polymerase III is a complex, multichain en...",DNA polymerase III (beta chain) DnaN (DNA nucl...,information pathways,Q50790,"GO:0003677,GO:0006260,GO:0003887,GO:0005737,GO...",NaN
2,Rv0003,recF,3280,4437,+,Non-Essential,CDS,The RECF protein is involved in DNA metabolism...,DNA replication and repair protein RecF (singl...,information pathways,Q59586,"GO:0006281,GO:0006260,GO:0009432,GO:0005737,GO...",NaN
3,Rv0004,Rv0004,4434,4997,+,Non-Essential,CDS,Function unknown,Conserved hypothetical protein,conserved hypotheticals,P71573,NaN,NaN
4,Rv0005,gyrB,5240,7267,+,Antibiotic Resistance,CDS,DNA gyrase negatively supercoils closed circul...,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,information pathways,P0C5C5,"GO:0003918,GO:0006265,GO:0005524",FLQ


In [14]:
dr_gene_df = h37rv_summary_df[h37rv_summary_df.gene_category_vargas2021 == "Antibiotic Resistance"]
print(dr_gene_df.shape)
dr_gene_df.head()

(28, 13)


,gene_id,gene_name,chrom_start,chrom_stop,strand,gene_category_vargas2021,feature,function,product,functional_category,PFAM,gene_ontology,dr_mutations
4,Rv0005,gyrB,5240,7267,+,Antibiotic Resistance,CDS,DNA gyrase negatively supercoils closed circul...,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,information pathways,P0C5C5,"GO:0003918,GO:0006265,GO:0005524",FLQ
5,Rv0006,gyrA,7302,9818,+,Antibiotic Resistance,CDS,DNA gyrase negatively supercoils closed circul...,DNA gyrase (subunit A) GyrA (DNA topoisomerase...,information pathways,Q07702,"GO:0003918,GO:0006265,GO:0046677,GO:0005524",FLQ
344,Rv0341,iniB,409362,410801,+,Antibiotic Resistance,CDS,Unknown,Isoniazid inductible gene protein IniB,cell wall and cell processes,NaN,NaN,"EMB,INH"
345,Rv0342,iniA,410838,412760,+,Antibiotic Resistance,CDS,Unknown,Isoniazid inductible gene protein IniA,cell wall and cell processes,O06293,"GO:0003924,GO:0016021,GO:0005886,GO:0046677,GO...","EMB,INH"
346,Rv0343,iniC,412757,414238,+,Antibiotic Resistance,CDS,Unknown,Isoniazid inductible gene protein IniC,cell wall and cell processes,O06294,"GO:0003924,GO:0005525","EMB,INH"


In [15]:
dr_gene_pos = {}

for i in dr_gene_df.index:
    
    # stop is inclusive
    chrom_start, chrom_stop = dr_gene_df.loc[i, "chrom_start"], dr_gene_df.loc[i, "chrom_stop"]
    
    gene_name, gene_id = dr_gene_df.loc[i, "gene_name"], dr_gene_df.loc[i, "gene_id"]
    
    for pos in range(chrom_start, chrom_stop+1):
        dr_gene_pos[pos] = (gene_name, gene_id)

In [16]:
with open("./data/DR_pos_geneIDName_dict.pkl", "wb") as out_f:
    pickle.dump(dr_gene_pos, out_f)

## Look at overlap between DR pos used for analysis and WHO DR pos used for simulation

In [2]:
with open("./data/DR_pos_geneIDName_dict.pkl", "rb") as in_f:
    dr_gene_pos = pickle.load(in_f)

In [3]:
dr_mutations = pd.read_csv("/home/sm624/data/antib_resistance/WHO_drug_SNP_mapping.csv")
dr_mutations

,gene_name,gene_locus,variant,codon_number,genome_index,nt_change,drug
0,gyrB,Rv0005,gyrB_D461N,461.0,6620,A,LEVO
1,gyrB,Rv0005,gyrB_D461N,461.0,6620,A,MOXI
2,gyrB,Rv0005,gyrB_N499D,499.0,6734,G,LEVO
3,gyrB,Rv0005,gyrB_N499D,499.0,6734,G,MOXI
4,gyrB,Rv0005,gyrB_E501V,501.0,6741,T,LEVO
...,...,...,...,...,...,...,...
508,gid,Rv3919c,gid_W45!,45.0,4408069,A,STR
509,gid,Rv3919c,gid_G37!,37.0,4408094,T,STR
510,gid,Rv3919c,gid_G28!,28.0,4408121,T,STR
511,gid,Rv3919c,gid_Y22!,22.0,4408137,A,STR


In [6]:
## how many were simulated actually

for i in dr_mutations.index:
    pos = dr_mutations.loc[i, "genome_index"]
    if pos not in dr_gene_pos:

        for mutant_num in range(1, 11):

            mutant_df = pd.read_csv("../../../mutant_summaries_final/mutant_{}.csv".format(mutant_num))
            mutant_df = mutant_df[mutant_df.TYPE == "DR"]

            if pos in mutant_df["POS"].values:
                print(mutant_num, pos)
        


3 801268
9 1472359
2 1472362
7 1472723
1 2715342
6 2715369
9 2715369
5 4243221
8 4243221
2 4327480
5 4327480


In [8]:
## how many were simulated actually

for i in dr_mutations.index:
    pos = dr_mutations.loc[i, "genome_index"]
    if pos not in dr_gene_pos:

        mutant_df = pd.read_csv("../../../lineage123/mutant_summaries_final/L123_mutant.csv")

        if pos in mutant_df["POS"].values:
            print(pos)
        


1473246
1473246
1473246


## Define LM positions

Define as all positions in windows in which every position has an EBR < 0.8 and a pileup mappability < 0.8

In [28]:
low_mappability_df = pd.read_csv("/home/sm624/projects/mixed_calls/benchmarking/comprehensive/h37rv_modification/low_mappability_regions.csv")
print(low_mappability_df.shape)
low_mappability_df.head()

(156, 2)


,start,stop
0,103950,104250
1,104100,104400
2,104250,104550
3,104400,104700
4,400500,400800


In [29]:
LM_pos = []
for i in low_mappability_df.index:
    
    start, stop = low_mappability_df.loc[i, "start"]+1, low_mappability_df.loc[i, "stop"]
    
    LM_pos += list(np.arange(start, stop))

In [30]:
with open("./data/LM_pos_list.pkl", "wb") as out_f:
    pickle.dump(LM_pos, out_f)

## Define HT positions

In the below CSV, the `H37Rv Start` index is 0-based and the HT extends to the `H37Rv End` index *inclusively*. So the HT starts at `H37Rv Start + 1` BUT the insertion will be captured in a VCF at `H37Rv Start` because the insertion will be logged at the beginning of the HT, appended to the base before the HT starts.

So, the positions at which we want to log HT's is actually just `H37Rv Start`.

In [2]:
homopolymer_df = pd.read_csv("/home/sm624/data/homopolymers/vargas_2023_supplement3_annotated.csv")
print(homopolymer_df.shape)
homopolymer_df

(145, 9)


,H37Rv Start,H37Rv End,polyNT,Locus Tag,Gene Symbol,Homoplasy Score,# Alternate,resistance_assocation,resistance_association_source
0,976897,976906,GGGGGGGGG,Rv0878c,PPE13,2317,8351,NaN,NaN
1,854252,854261,CCCCCCCCC,inter_Rv0759c_Rv0760c,inter_Rv0759c_Rv0760c,776,28077,Y,(Vargas et al. 2023)
2,976889,976896,TTTTTTTT,Rv0878c,PPE13,771,5641,NaN,NaN
3,1992323,1992331,CCCCCCCC,Rv1759c,wag22,578,4052,NaN,NaN
4,2338194,2338202,CCCCCCCC,Rv2081c,Rv2081c,360,4596,NaN,NaN
...,...,...,...,...,...,...,...,...,...
140,2858258,2858265,CCCCCCC,Rv2533c,nusB,0,0,NaN,NaN
141,1091980,1091987,GGGGGGG,Rv0977,PE_PGRS16,0,0,NaN,NaN
142,1203483,1203490,GGGGGGG,Rv1078,Rv1078,0,0,NaN,NaN
143,2804942,2804949,CCCCCCC,Rv2490c,PE_PGRS43,0,0,NaN,NaN


In [3]:
homopolymer_df.loc[145] = [779181, 779187, "GGGGGG", "Rv0678", "Rv0678", np.nan, np.nan, "Y", "(Xu et al. 2023)"]

In [4]:
print(homopolymer_df.shape)
homopolymer_df

(146, 9)


,H37Rv Start,H37Rv End,polyNT,Locus Tag,Gene Symbol,Homoplasy Score,# Alternate,resistance_assocation,resistance_association_source
0,976897,976906,GGGGGGGGG,Rv0878c,PPE13,2317.0,8351.0,NaN,NaN
1,854252,854261,CCCCCCCCC,inter_Rv0759c_Rv0760c,inter_Rv0759c_Rv0760c,776.0,28077.0,Y,(Vargas et al. 2023)
2,976889,976896,TTTTTTTT,Rv0878c,PPE13,771.0,5641.0,NaN,NaN
3,1992323,1992331,CCCCCCCC,Rv1759c,wag22,578.0,4052.0,NaN,NaN
4,2338194,2338202,CCCCCCCC,Rv2081c,Rv2081c,360.0,4596.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
141,1091980,1091987,GGGGGGG,Rv0977,PE_PGRS16,0.0,0.0,NaN,NaN
142,1203483,1203490,GGGGGGG,Rv1078,Rv1078,0.0,0.0,NaN,NaN
143,2804942,2804949,CCCCCCC,Rv2490c,PE_PGRS43,0.0,0.0,NaN,NaN
144,1573160,1573167,CCCCCCC,Rv1396c,PE_PGRS25,0.0,0.0,NaN,NaN


In [11]:
homopolymer_df["H37Rv Start"].value_counts()

2517589    2
2571311    1
2469149    1
1860939    1
1852335    1
          ..
3832356    1
4026422    1
373026     1
1685259    1
779181     1
Name: H37Rv Start, Length: 145, dtype: int64

In [7]:
HT_pos = {}

for i in homopolymer_df.index:
    
    var_pos, base = homopolymer_df.loc[i, "H37Rv Start"], homopolymer_df.loc[i, "polyNT"][0]
    
    HT_pos[var_pos] = base

In [6]:
len(HT_pos)

145

Even though DataFrame has 146 entries, one position is duplicated:
```
2517589,2517596,CCCCCCC,inter_RVnc0018_Rv2244,inter_mcr16_acpM,0,0,,
2517589,2517596,CCCCCCC,Rv2243,fabD,0,0,,
```

In [6]:
with open("./data/HT_pos_varBase_dict.pkl", "wb") as out_f:
    pickle.dump(HT_pos, out_f)